In [ ]:
# pip install fitz
# pip install PyMuPDF
# pip install kss

In [3]:
import os
present_dir = os.getcwd()
parent_dir = os.path.dirname(present_dir)
lawdata_dir = parent_dir + '\\crawl_report\\'
print('현재경로 :', present_dir)
print('상위경로 :', parent_dir)
print('크롤링데이터 경로 :', lawdata_dir)

현재경로 : c:\Users\SDA05\Desktop\work\BOK_project_TEAM2\hsunnie\pre_processing_report
상위경로 : c:\Users\SDA05\Desktop\work\BOK_project_TEAM2\hsunnie
크롤링데이터 경로 : c:\Users\SDA05\Desktop\work\BOK_project_TEAM2\hsunnie\crawl_report\


In [5]:
import os
import fitz
import pandas as pd
import kss
from ekonlpy.tag import Mecab

def txt2filtered(text):
    tokenizer = Mecab()
    tokens = tokenizer.pos(text)
    filtered_tokens = [token[0] for token in tokens if token[1] in ['NNG', 'VA', 'VAX', 'MAG', 'VA']]
    return filtered_tokens

df = pd.read_csv(lawdata_dir+"pdf_link_crawl_add_txt.csv", sep='\t')
pdf_dir = lawdata_dir + 'reportpdf/'
txt_dir = './reporttxt_token/'
if not os.path.exists(txt_dir):
    os.makedirs(txt_dir)

for i in range(len(df)):
    df_info_list = df.iloc[i].to_list()
    pdf_name = pdf_dir + df_info_list[3]
    txt_name = txt_dir + df_info_list[4]#.replace('.pdf', '.txt')
    
    # 파일이 이미 존재하는 경우에는 생성하지 않음
    if not os.path.exists(txt_name):
        with fitz.open(pdf_name) as doc:
            all_filtered_sentences = []
            for page in doc:
                content = page.get_text()
                sentences = kss.split_sentences(content)
                for sentence in sentences:
                    filtered_tokens = txt2filtered(sentence)
                    if filtered_tokens:
                        all_filtered_sentences.append(filtered_tokens)
                        
            with open(txt_name, 'w', encoding='utf-8') as f:
                f.write(str(all_filtered_sentences))
